In [10]:
import pandas as pd
import numpy as np

In [11]:
df = pd.read_csv('covid.csv')
df

,age,gender,fever,cough,city,has_covid,Unnamed: 6
0,NaN,60,Male,103.0,Mild,Kolkata,No
1,NaN,27,Male,100.0,Mild,Delhi,Yes
2,NaN,42,Male,101.0,Mild,Delhi,No
3,NaN,31,Female,98.0,Mild,Kolkata,No
4,NaN,65,Female,101.0,Mild,Mumbai,No
...,...,...,...,...,...,...,...
93,NaN,27,Male,100.0,Mild,Kolkata,Yes
94,NaN,79,Male,NaN,Strong,Kolkata,Yes
95,NaN,12,Female,104.0,Mild,Bangalore,No
96,NaN,51,Female,101.0,Strong,Kolkata,Yes


In [12]:
df['age']

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
      ..
93   NaN
94   NaN
95   NaN
96   NaN
97   NaN
Name: age, Length: 98, dtype: float64

In [7]:
df.drop('age', axis=1, inplace=True)

In [8]:
df = df.rename(columns={'gender':'age', 'fever':'gender','cough':'fever','city':'cough','has_covid':'city','Unnamed: 6':'has_covid'})

In [9]:
df

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No
...,...,...,...,...,...,...
93,27,Male,100.0,Mild,Kolkata,Yes
94,79,Male,NaN,Strong,Kolkata,Yes
95,12,Female,104.0,Mild,Bangalore,No
96,51,Female,101.0,Strong,Kolkata,Yes


In [11]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [12]:
#fill missing values in fever column with the mean with SimpleImputer

In [19]:
df[df['fever'].isnull()]

,age,gender,fever,cough,city,has_covid
5,84,Female,NaN,Mild,Bangalore,Yes
7,20,Female,NaN,Strong,Mumbai,Yes
10,75,Female,NaN,Mild,Delhi,No
19,42,Female,NaN,Strong,Bangalore,Yes
25,23,Male,NaN,Mild,Mumbai,No
29,34,Female,NaN,Strong,Mumbai,Yes
41,82,Male,NaN,Mild,Kolkata,Yes
56,71,Male,NaN,Strong,Kolkata,No
91,38,Male,NaN,Mild,Delhi,Yes
94,79,Male,NaN,Strong,Kolkata,Yes


In [13]:
from sklearn.impute import SimpleImputer

In [22]:
si = SimpleImputer(missing_values=np.nan,strategy='mean')

In [24]:
X_fever = si.fit_transform(df[['fever']])

In [28]:
X_fever = np.round(X_fever.astype('float'),2)

In [31]:
#applying OHE for gender and city 

In [39]:
df[['gender','city']]

,gender,city
0,Male,Kolkata
1,Male,Delhi
2,Male,Delhi
3,Female,Kolkata
4,Female,Mumbai
...,...,...
93,Male,Kolkata
94,Male,Kolkata
95,Female,Bangalore
96,Female,Kolkata


In [40]:
df['city'].value_counts()

Kolkata      31
Bangalore    30
Delhi        22
Mumbai       15
Name: city, dtype: int64

In [ ]:
#so there will be 1 coulmn for gender and 3 column for city after dropping first

In [33]:
from sklearn.preprocessing import OneHotEncoder

In [34]:
ohe = OneHotEncoder(sparse=False, drop='first')

In [36]:
X_gender_city = ohe.fit_transform(df[['gender','city']])

In [41]:
X_gender_city.shape

(98, 4)

In [49]:
df['cough'].head(10)

0      Mild
1      Mild
2      Mild
3      Mild
4      Mild
5      Mild
6    Strong
7    Strong
8    Strong
9      Mild
Name: cough, dtype: object

In [ ]:
#applying ordinal encoding for cough column : low value for Mild and high value for Strong

In [44]:
from sklearn.preprocessing import OrdinalEncoder

In [47]:
ordl = OrdinalEncoder(categories=[['Mild','Strong']])

In [50]:
X_cough = ordl.fit_transform(df[['cough']])

In [53]:
type(X_cough)

numpy.ndarray

In [51]:
#preparing our final dataset

In [64]:
df_final = np.concatenate((X_cough, X_fever, X_gender_city), axis=1)

In [98]:
df_final.shape

(98, 6)

In [70]:
df_final[0:10,]

array([[  0.  , 103.  ,   1.  ,   0.  ,   1.  ,   0.  ],
       [  0.  , 100.  ,   1.  ,   1.  ,   0.  ,   0.  ],
       [  0.  , 101.  ,   1.  ,   1.  ,   0.  ,   0.  ],
       [  0.  ,  98.  ,   0.  ,   0.  ,   1.  ,   0.  ],
       [  0.  , 101.  ,   0.  ,   0.  ,   0.  ,   1.  ],
       [  0.  , 100.91,   0.  ,   0.  ,   0.  ,   0.  ],
       [  1.  , 101.  ,   1.  ,   0.  ,   0.  ,   0.  ],
       [  1.  , 100.91,   0.  ,   0.  ,   0.  ,   1.  ],
       [  1.  , 100.  ,   0.  ,   0.  ,   0.  ,   0.  ],
       [  0.  , 101.  ,   0.  ,   1.  ,   0.  ,   0.  ]])

In [ ]:
#feature engineering using ColumnTransformer in one step instead of all above step

In [72]:
from sklearn.compose import ColumnTransformer

In [ ]:
X = df.drop('has_covid', axis=1)
X

In [94]:
cl = ColumnTransformer( transformers = [('lf1',SimpleImputer(),['fever']),
                       ('lf2', OneHotEncoder(drop='first', sparse=False),['gender','city']),
                       ('lf3', OrdinalEncoder(categories=[['Mild','Strong']]),['cough'])
                                       ])


In [96]:
X_final = cl.fit_transform(X)

In [97]:
X_final.shape

(98, 6)

In [99]:
X_final[0:10,]

array([[103.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ],
       [100.        ,   1.        ,   1.        ,   0.        ,
          0.        ,   0.        ],
       [101.        ,   1.        ,   1.        ,   0.        ,
          0.        ,   0.        ],
       [ 98.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   0.        ],
       [101.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ],
       [100.90909091,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [101.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   1.        ],
       [100.90909091,   0.        ,   0.        ,   0.        ,
          1.        ,   1.        ],
       [100.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   1.        ],
       [101.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0. 